<a href="https://colab.research.google.com/github/tfukuda675/gee_folium_overview/blob/master/tweet_streaming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 参考URL

https://akitoshiblogsite.com/tweepy-streaming-api/


https://kurozumi.github.io/tweepy/streaming_how_to.html


## twitter API公式ドキュメント
https://developer.twitter.com/en/docs/twitter-api/tweets/filtered-stream/introduction
サンプルコードも公開
https://github.com/twitterdev/Twitter-API-v2-sample-code


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!mkdir /root/.twitter
!cp drive/My\ Drive/twitter.json /root/.twitter/.
!chmod 600 /root/.twitter/twitter.json

mkdir: cannot create directory ‘/root/.twitter’: File exists


In [3]:
!pip install tweepy

In [4]:
import tweepy
import json
import inspect
import subprocess
import time
import datetime
import folium
import sqlite3
import pandas as pd
from os.path import expanduser
from numba import jit

## Class
override tweepy.StreamListener to add logic to on_status

In [6]:
class MyStreamListener(tweepy.StreamListener):

    def __init__(self):
        # limit running  
        self.stTime = time.time()
        self.__limit = 60
        self.__monitortime  =   datetime.datetime.now().strftime('%Y/%m/%d/ %H:%M')
        self.__count = int()
        self.__sqlite3db    =   str()
        self.__sqlite3_conn =   None
        self.__sqlite3_cur  =   None
        self.__datadict     =   dict()
        self.__pickup_words =   list()
        super(MyStreamListener, self).__init__()

    def on_status(self, status):
        now_for_db = datetime.datetime.now().strftime('%Y/%m/%d/ %H:%M')
        if now_for_db == self.__monitortime:
            text = str()
            if not status.truncated:
                text = status.text
            else:
                text = status.extended_tweet['full_text']
            for w in self.__pickup_words:
                if w in text:
                    if debug_mode:
                        print("HIT!!!!!!!!!!!!!!!!!!!")
                    self.__datadict[w] += 1
        else:
            self.__monitortime = now_for_db
            print("\n##====================")
            print(now_for_db)
            text = str()
            if not status.truncated:
                text = status.text
            else:
                text = status.extended_tweet['full_text']
            
            for w in self.__pickup_words:
                if w in text:
                    if debug_mode:
                        print("HIT!!!!!!!")
                    self.__datadict[w] += 1

            #      ___________________________________________
            #_____/ Store result data                         \______________
            #
            print(self.__datadict)
            print("##====================\n")
            print(list(self.__datadict.keys()))
            print(list(self.__datadict.values()))
            df = pd.DataFrame(columns=list(self.__datadict.keys()))
            df.loc[now_for_db] = list(self.__datadict.values())
            df.to_sql("pickup_table",self.__sqlite3_conn,if_exists='append')

            #      ___________________________________________
            #_____/ Initialize data                           \______________
            #
            for w in self.__pickup_words:
                self.__datadict[w] = int(0)
    
        if(time.time() - self.stTime) > self.__limit:
            print("¥n¥nKIC-Info :: Finished")
            self.sqlite3_close()
            return(False)
        print("name : %s, screen_name : %s" % (status.user.name,status.user.screen_name))
        print("time : {}".format(now_for_db))
        if not status.truncated:
            print("text : %s " % status.text)
        else:
            print("full_text : {}".format(status.extended_tweet['full_text']))

        print("-"*50)

        self.__count += 1

    @property
    def limit_time(self):
        return self.__limit

    @limit_time.setter
    def limit_time(self,limit):
        self.__limit = limit

    @property
    def sqlite3db(self):
        return self.__sqlite3db

    @sqlite3db.setter
    def sqlite3db(self,db):
        self.__sqlite3db    =   db
        self.__sqlite3_conn =   sqlite3.connect(self.__sqlite3db)
        self.__sqlite3_cur  =   self.__sqlite3_conn.cursor()

    def sqlite3_close(self):
        self.__sqlite3_cur.close()
        self.__sqlite3_conn.close()

    @property
    def pickup_words(self):
        return self.__pickup_words

    @pickup_words.setter
    def pickup_words(self,words):
        self.__pickup_words = words
        for w in self.__pickup_words:
            self.__datadict[w] = int(0)

    @property
    def count(self):
        return self.__count

In [8]:
config = json.load(open("/root/.twitter/twitter.json","r"))
auth    =   tweepy.OAuthHandler(config["api_key"], config["api_secret_key"])
auth.set_access_token(config["access_token"], config["access_token_secret"])
api     =   tweepy.API(auth, wait_on_rate_limit = True)
debug_mode   =   True


count = 5
pickup_words = ['Python','キオクシア','KIOXIA']
sqlite3_db = "./tweet_store.sqlite3"

for i in range(count):
    td = datetime.timedelta(minutes=1)
    myStreamListener = MyStreamListener()
    myStreamListener.limit_time = td.total_seconds()
    myStreamListener.sqlite3db = sqlite3_db
    myStreamListener.pickup_words = pickup_words
    myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener,tweet_mode='extended')
    # filter word 
    myStream.filter(track=pickup_words)
    print("KIC-Info ::  Tweet count")
    print(myStreamListener.count)

HIT!!!!!!!!!!!!!!!!!!!
name : CodeSprintPro, screen_name : sprint_code
time : 2021/01/11/ 12:20
text : RT @Analytics_699: Predicting the decision of CNN via SAURON (XAI) ▶️
#Analytics #BigData #Rstats #AI #Reactjs #ML #NLP #Python
#MachineLea… 
--------------------------------------------------
name : The Fiery Bot, screen_name : TheFieryBot
time : 2021/01/11/ 12:20
text : RT @Tanguy_lrx: Sad to say but #covid was a way to move things...

#hclswlobp #cloud #Blockchain #nocode #lowcode #javascript #github #node… 
--------------------------------------------------
HIT!!!!!!!!!!!!!!!!!!!
name : IoT (Internet of Things), screen_name : IoTinternet
time : 2021/01/11/ 12:20
text : RT @Eli_Krumova: Learn #Python for #DataScience with #Excel: Summarize #Data in #Pandas
👇🏻
https://t.co/hnXrDMNfH0
v/ @SourabhSKatoch

#Mac… 
--------------------------------------------------
name : 拡散⚽️Botくん, screen_name : kazoonBot
time : 2021/01/11/ 12:20
text : RT @keimy_engineer: 【YouTube拡散希望】

python3のチップス
at

In [9]:
conn = sqlite3.connect(sqlite3_db)
df=pd.read_sql_query('SELECT * FROM pickup_table', conn)
print(df)
conn.close()

               index  Python  キオクシア  KIOXIA
0  2021/01/11/ 12:21      21      0       0
1  2021/01/11/ 12:22      13      0       0
2  2021/01/11/ 12:23       9      0       0
3  2021/01/11/ 12:24      15      0       0
4  2021/01/11/ 12:25      20      0       0


In [ ]:
# see the source code of tweepy.StreamListener
#path = inspect.getfile(tweepy.StreamListener)
#subprocess.check_call(["open",path ])
#print(path)

FileNotFoundError: ignored

In [ ]:
print(json.dumps(api.rate_limit_status(),indent=4))

{
    "rate_limit_context": {
        "access_token": "1327250356259098624-R3efgk3v3KzdSROVqbcuCnmqPGDols"
    },
    "resources": {
        "lists": {
            "/lists/list": {
                "limit": 15,
                "remaining": 15,
                "reset": 1609160775
            },
            "/lists/memberships": {
                "limit": 75,
                "remaining": 75,
                "reset": 1609160775
            },
            "/lists/subscribers/show": {
                "limit": 15,
                "remaining": 15,
                "reset": 1609160775
            },
            "/lists/members": {
                "limit": 900,
                "remaining": 900,
                "reset": 1609160775
            },
            "/lists/subscriptions": {
                "limit": 15,
                "remaining": 15,
                "reset": 1609160775
            },
            "/lists/show": {
                "limit": 75,
                "remaining": 75,
              